In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm
tqdm().pandas()
import numpy as np

from collections import Counter

from sklearn.metrics.pairwise import cosine_similarity

path = 'D:/Users/Nicola Melluso/ScientificPublications/'
path_vectors = path + 'Papers/TitleAbstract/papers_vectors/'

path_processed = path + 'PapersProcessed/'

gram = 'trigrams'

0it [00:00, ?it/s]

## ************** 

## Preliminaries

### 1. Papers to be done

In [2]:
reader = open('./data/paperIds.txt','r')
paperIds = set()

for line in tqdm(reader,total = 72245396):
    paperIds.add(int(line.replace('\n','')))
    

reader = open('./data/paperIdsNov.txt','r')
paperIdsNov = set()

for line in tqdm(reader,total = 33536292):
    paperIdsNov.add(int(line.replace('\n','')))

  0%|          | 0/72245396 [00:00<?, ?it/s]

  0%|          | 0/33536292 [00:00<?, ?it/s]

### 2. Gram to idx 

In [3]:
reader = open('./data/%s_to_idx.csv'%(gram.split('_')[0]),'r', encoding="utf-8")
reader.__next__()

gram_to_idx = {}

for line in tqdm(reader):
    
    if ',' not in line:
        continue
        
    line = line.split(',')
    gram_to_idx[line[0]] = int(line[1].replace('\n',''))

0it [00:00, ?it/s]

### 3. Core analysis

In [4]:
print('Reading baseline...')
baseline = set()

reader = open(path_processed + '%s_baseline.txt'%(gram),'r', encoding="utf-8")

for line in tqdm(reader):
    
    t = gram_to_idx.get(line.replace('\n',''))
    if t is not None:
        baseline.add(t)
        
print('Analysis...')

reader = open(path_processed + '%s.csv'%(gram.split('_')[0]),'r', encoding="utf-8")
reader.__next__()

writer = open('./data/new_%s/new_%s_paperId.csv'%(gram,gram),'w', encoding="utf-8")
writer.write('gram_idx,PaperId\n')

counter = Counter()

for line in tqdm(reader, total = 103061513):
    
    line = line.split('\t')
    
    paperId = int(line[0])
    
    if paperId not in paperIds:
        continue
                
    text = set(' '.join(line[1:]).split(' '))
    #text = set(line[1].split(' '))
    text = [gram_to_idx.get(t) for t in text]
    text = [t for t in text if ((t is not None) & (t not in baseline))]
    
    for token in text:
        if token not in counter:
            if paperId not in paperIdsNov:
                continue
            
            counter[token] = 0
            
            ## write the PaperId
            writer.write(','.join([
                str(i) for i in [
                    token,
                    paperId
                ]
            ]) + '\n')
        else:
            counter[token] += 1
    
print('Exporting the counter...')
writer = open('./data/new_%s/new_%s_reuse.csv'%(gram,gram),'w', encoding="utf-8")
writer.write('gram_idx,reuse\n')

for token,reuse in tqdm(counter.items(), total = len(counter)):
    writer.write(','.join([
                str(i) for i in [
                    token,
                    reuse
                ]
            ]) + '\n')

Reading baseline...


0it [00:00, ?it/s]

Analysis...


  0%|          | 0/103061513 [00:00<?, ?it/s]

Exporting the counter...


  0%|          | 0/6140337 [00:00<?, ?it/s]

## 2. Adjust non-doi and citations minimum count 

#### Read grams with only reuse < 5 

In [5]:
counter = set()
tokens = set()

reader = open('./data/new_%s/new_%s_reuse.csv'%(gram,gram),'r', encoding="utf-8")
reader.__next__()

for line in tqdm(reader):
    line = line.split(',')
    
    reuse = int(line[1].replace('\n',''))
    
    if reuse <= 5:
        continue
        
    token = int(line[0])
    
    tokens.add(token)

0it [00:00, ?it/s]

`tokens` contains all tokens with reuse > 5.

### Re-do analysis 

Create a new file `new_gram_PaperId_DC.csv` with the real PaperIds.

In [6]:
print('Reading baseline...')
baseline = set()
dones = set()
counter = Counter()

reader = open(path_processed + '%s_baseline.txt'%(gram),'r', encoding="utf-8")

for line in tqdm(reader):
    
    t = gram_to_idx.get(line.replace('\n',''))
    if t is not None:
        baseline.add(t)
        
print('Analysis...')

reader = open(path_processed + '%s.csv'%(gram.split('_')[0]),'r', encoding="utf-8")
reader.__next__()

writer = open('./data/new_%s/new_%s_paperId_DC.csv'%(gram,gram),'w', encoding="utf-8")
writer.write('gram_idx,PaperId\n')


for line in tqdm(reader, total = 103061513):
    
    line = line.split('\t')
    
    paperId = int(line[0])
    
    if paperId not in paperIds:
        continue
                
    text = set(' '.join(line[1:]).split(' '))
    #text = set(line[1].split(' '))
    text = [gram_to_idx.get(t) for t in text]
    text = [t for t in text if ((t is not None) & (t not in baseline))]
    
    for token in text:
        if token not in tokens:
            if token not in counter:
            
                ## write the PaperId real
                writer.write(','.join([
                    str(i) for i in [
                        token,
                        paperId
                    ]
                ]) + '\n')
                
                counter[token] = 0
            else:
                counter[token] += 1

writer.close()
reader.close()


print('Exporting the counter...')
writer = open('./data/new_%s/new_%s_reuse_DC.csv'%(gram,gram),'w', encoding="utf-8")
writer.write('gram_idx,reuse\n')

for token,reuse in tqdm(counter.items(), total = len(counter)):
    writer.write(','.join([
                str(i) for i in [
                    token,
                    reuse
                ]
            ]) + '\n')
    
writer.close()


Reading baseline...


0it [00:00, ?it/s]

Analysis...


  0%|          | 0/103061513 [00:00<?, ?it/s]

Exporting the counter...


  0%|          | 0/4118839 [00:00<?, ?it/s]

# ********* Re-start kernel ********** 

# Aggregations 
### To execute after the counter

Chunking strategy 

In [2]:
reader = open('./data/%s_to_idx.csv'%(gram.replace('_title','')),'r', encoding="utf-8")
reader.__next__()

idx_to_gram = {}

for line in tqdm(reader):
    
    if ',' not in line:
        continue
        
    line = line.split(',')
    idx_to_gram[int(line[1].replace('\n',''))] = line[0]

0it [00:00, ?it/s]

In [3]:
chunksize = 500000

ds_paperId = pd.read_csv('./data/new_%s/new_%s_paperId.csv'%(gram,gram), chunksize = chunksize, converters = {'PaperId':int,'gram_idx':int}, encoding="utf-8")

writer = open('./data/new_%s/new_%s.csv'%(gram,gram),'w', encoding="utf-8")
writer.write('%s,PaperId,reuse\n'%(gram))
writer.close()


print('First iterate over all...')
# iterate over paperIds
for df_paperId in tqdm(ds_paperId):
    
    
    to_do_ids = list(df_paperId['PaperId'].unique())
    
    if to_do_ids == set():
        continue

    ds_reuse = pd.read_csv('./data/new_%s/new_%s_reuse.csv'%(gram,gram), chunksize = chunksize, converters = {'PaperId':int,'gram_idx':int}, encoding="utf-8")

    for df_reuse in ds_reuse:
        
        df_reuse = df_reuse[df_reuse['reuse'] > 5]
        
        df = pd.merge(df_paperId,df_reuse, on = 'gram_idx')
                    
        if df.empty:
            continue

        df[gram] = df['gram_idx'].apply(lambda x: idx_to_gram.get(x))
        df = df[[gram,'PaperId','reuse']]
        df.to_csv('./data/new_%s/new_%s.csv'%(gram,gram), mode = 'a', header = False, index = False)
        
        dones_ids = set(df['PaperId'].unique())
        
        to_do_ids = to_do_ids - dones_ids
        
        if to_do_ids == set():
            break

    
print('Second iterate over non-DOI and citation minimum...')

ds_paperId_DC = pd.read_csv('./data/new_%s/new_%s_paperId_DC.csv'%(gram,gram), chunksize = chunksize, converters = {'PaperId':int,'gram_idx':int},  encoding="utf-8")

for df_paperId_DC in tqdm(ds_paperId_DC):
    
    to_do_ids = list(df_paperId_DC['PaperId'].unique())
    
    if to_do_ids == set():
        continue

    ds_reuse = pd.read_csv('./data/new_%s/new_%s_reuse_DC.csv'%(gram,gram), chunksize = chunksize, converters = {'PaperId':int,'gram_idx':int}, encoding="utf-8")

    for df_reuse in ds_reuse:
        
        df_reuse = df_reuse[df_reuse['reuse'] > 0]
        
        df = pd.merge(df_paperId_DC,df_reuse, on = 'gram_idx')
        
        
        if df.empty:
            continue
        
        df[gram] = df['gram_idx'].apply(lambda x: idx_to_gram.get(x))
        df = df[[gram,'PaperId','reuse']]
        df.to_csv('./data/new_%s/new_%s.csv'%(gram,gram), mode = 'a', header = False, index = False)  
        
        dones_ids = set(df['PaperId'].unique())
        
        to_do_ids = to_do_ids - dones_ids
        
        if to_do_ids == set():
            break



First iterate over all...


0it [00:00, ?it/s]

Second iterate over non-DOI and citation minimum...


0it [00:00, ?it/s]

### Memory intensive 

In [4]:
gram_label = 'trigrams'
gram = 'trigrams'

print('First iteration...')

print('\tReading combs reuse...')
df_reuse = pd.read_csv('./data/new_%s/new_%s_reuse.csv'%(gram,gram), converters={'PaperId': int, 'gram_idx':int,'reuse': int},  encoding="utf-8")
df_reuse = df_reuse[df_reuse['reuse'] > 5]

print('\tReading combs paperId...')
df_paperId = pd.read_csv('./data/new_%s/new_%s_paperId.csv'%(gram,gram), converters = {'PaperId':int, 'gram_idx':int},  encoding="utf-8")
print('\tConverting PaperId...')

print('\tMerging...')
df = pd.merge(df_paperId,df_reuse, on = ['gram_idx'])

print('\tConverting grams...')
df[gram] = df['gram_idx'].apply(lambda x: idx_to_gram.get(x))

df = df[[gram,'PaperId','reuse']]

print('\tExporting...')
df.to_csv('./data/new_%s/new_%s.csv'%(gram,gram), index = False)


print('Second iteration...')

print('\tReading combs reuse...')
df_reuse = pd.read_csv('./data/new_%s/new_%s_reuse_DC.csv'%(gram,gram), converters={'PaperId': int, 'gram_idx':int,'reuse': int},  encoding="utf-8")
df_reuse = df_reuse[df_reuse['reuse'] > 0]

print('\tReading combs paperId...')
df_paperId = pd.read_csv('./data/new_%s/new_%s_paperId_DC.csv'%(gram,gram), converters = {'PaperId':int, 'gram_idx':int},  encoding="utf-8")
print('\tConverting PaperId...')

print('\tMerging...')
df = pd.merge(df_paperId,df_reuse, on = ['gram_idx'])

print('\tConverting grams...')
df[gram] = df['gram_idx'].apply(lambda x: idx_to_gram.get(x))

df = df[[gram,'PaperId','reuse']]

print('\tExporting...')
df.to_csv('./data/new_%s/new_%s.csv'%(gram,gram), mode = 'a', header = False, index = False)


First iteration...
	Reading combs reuse...
	Reading combs paperId...
	Converting PaperId...
	Merging...
	Converting grams...
	Exporting...
Second iteration...
	Reading combs reuse...
	Reading combs paperId...
	Converting PaperId...
	Merging...
	Converting grams...
	Exporting...


In [6]:
df = pd.read_csv('./data/new_%s/new_%s.csv'%(gram,gram))
df = df.sort_values(by = 'reuse', ascending = False)
df

,trigrams,PaperId,reuse
961665,quality_of_life,2215226054,346190
1256843,scanning_electron_microscopy,2058151030,269631
1657731,magnetic_resonance_imaging,69299825,250121
616325,transmission_electron_microscopy,1973732174,231922
4887036,polymerase_chain_reaction,2148312059,209968
...,...,...,...
43367846,time_to_comp,2034131755,1
43367849,arithmetic_and_orthogonal,2098195831,1
43367855,defend_the_sort,2142018294,1
43367856,reduce_chemical_control,2012308680,1


In [9]:
df[:10]

,trigrams,PaperId,reuse
961665,quality_of_life,2215226054,346190
1256843,scanning_electron_microscopy,2058151030,269631
1657731,magnetic_resonance_imaging,69299825,250121
616325,transmission_electron_microscopy,1973732174,231922
4887036,polymerase_chain_reaction,2148312059,209968
2074922,randomize_controlled_trial,2004769130,123107
239290,compare_to_control,2321278216,115503
15224,squamous_cell_carcinoma,2324369671,114257
35756,patient_with_acute,1992093885,114109
2108662,fourier_transform_infrared,1966994274,105576


In [8]:
df.to_csv('./data/new_%s/new_%s.csv'%(gram,gram), index = False)